In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

##### Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

class ModelTrainerConfig:
    root_dir: Path
    x_train_data_path : Path
    y_train_data_path : Path
    model_name : str
    c : float
    penalty : str

#### Configuration Manager

In [ ]:
from lone_approval_prediction.constants import *
from lone_approval_prediction.utils.comman import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.confit.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.logisticRegressor
        schema = self.schema.TARGET_COLUMN
        
        creat_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            x_train_data_path = config.x_train_data_path,
            y_train_data_path = config.y_train_data_path,
            model_name = config.model_name,
            c = params.c,
            penalty = params.penalty,
            )
        
        return model_trainer_config

#### Component

In [8]:
import os
import pandas as pd
import joblib
from lone_approval_prediction import logger
from sklearn.linear_model import LogisticRegression

class ModelTrainer:
    def __init__(self, config:ModelTrainerconfig) :
        self.config = config
        
    def train_model(self):
        x_train = pd.read_csv(self.config.x_train_data_path)
        y_train = pd.read_csv(self.config.y_train_data_path)
        
        logger.info(f"model trainig has started")
        logistic_regressor = LogisticRegression(penalty=self.config.penalty,c=self.config.c)
        logistic_regressor.fit(x_train,y_train)
        logger.info(f"model trainig has completed")
        
        joblib.dump(logistic_regressor,os.path.join(self.config.root_dir,self.config.model_name))
        
        logger.info(f"model has been saved at {self.config.root_dir}")
        
    

#### Pipeline

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train_model()
except Exception as e:
    raise e
    

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'